# Exercise 8 Dehalogenation on metals   9.4.2025


**Submission deadline**: End of May 2025

In this exercise we will take inspiration from the paper by Chen and collaborators  

[A Density Functional Benchmark for Dehydrogenation and
Dehalogenation Reactions on Coinage Metal Surfaces](http://doi.org/10.1002/cphc.202400865).

Many things have been done in that paper, including comparison of molecular adsorption energies, structure, dehydrogenations and dehalogenation (with reaction energy and barriers), different substrates, different xc functionals.

Today we will perform a small subset

* Generate small Cu and Au slabs
* Optimize bromobenzene in the gas phase
* Compute the energy of slabs and optimize slab+molecules systems (on Cu)
* Compute reaction energy for debromination at the PBE+D3 level starting from the paper optimized geometries on Cu and Au
* Compute reaction energy with another functional.



In [ ]:
#
# Preliminaries
#

#
# some important Imports...
#
import numpy as np
from ase import Atoms
from ase.io import read, write
from ase.visualize import view
import matplotlib.pyplot as plt
import nglview as nv
from ase.build import molecule
from ase.calculators.cp2k import CP2K
from ase.optimize import BFGS
from ase.io.trajectory import Trajectory
from ase.constraints import FixAtoms
from ase.build import surface, bulk
from ase.visualize import view

#%load_ext aiida
#%aiida
#from aiida import orm


#
# and definitions of visualization functions (see last exercises)
#
def view_structure(structure,myvec=[]):
    t = nv.ASEStructure(structure)
    w = nv.NGLWidget(t, gui=True)
    w.add_unitcell()
    w.add_spacefill(radius=0.2)
#    w.add_ball_and_stick()
    w.add_representation('label',label_type='atomindex',color='black')
#    w.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
#    w.add_representation('spacefill',color="blue",radius=0.5)

    return w

def view_trajectory(trajectory,myvec=[]):
    t2 = nv.ASETrajectory(trajectory)
    w2 = nv.NGLWidget(t2, gui=True)
    w2.add_representation('label',label_type='atomindex',color='black')
    #w2.add_unitcell()
    w2.add_ball_and_stick()
    w2.add_representation('spacefill',selection=myvec,color="red",radius=0.5)
    return w2

# Initialisation of the dictionaries

In [ ]:
calc = {}
filetraj = {}
energy = {}


# Creation of calculator

In [ ]:
inp_pbed3="""&FORCE_EVAL
 &DFT
   BASIS_SET_FILE_NAME BASIS_MOLOPT
   BASIS_SET_FILE_NAME BASIS_MOLOPT_UZH
 CHARGE 0
 &QS
   METHOD GPW                       !use gaussians and plane waves
 &END QS
    &SCF
      &OT
        PRECONDITIONER FULL_SINGLE_INVERSE
        MINIMIZER DIIS
        N_DIIS 7
      &END
      SCF_GUESS ATOMIC
      EPS_SCF 2.0E-5
      MAX_SCF 100
       &OUTER_SCF
          EPS_SCF 2.0E-5
          MAX_SCF 100
       &END
    &END SCF
 &XC
        &VDW_POTENTIAL
            DISPERSION_FUNCTIONAL PAIR_POTENTIAL
            &PAIR_POTENTIAL
               CALCULATE_C9_TERM .TRUE.
               PARAMETER_FILE_NAME dftd3.dat
               REFERENCE_FUNCTIONAL PBE
               R_CUTOFF 15
               TYPE DFTD3
            &END PAIR_POTENTIAL
         &END VDW_POTENTIAL
 &END XC
 &END DFT
 &SUBSYS
   &KIND Au
     BASIS_SET SZV-MOLOPT-SR-GTH
     POTENTIAL GTH-PBE-q11
   &END KIND
   &KIND Cu
     BASIS_SET SZV-MOLOPT-SR-GTH
    POTENTIAL GTH-PBE-q11
   &END KIND
   &KIND C
      BASIS_SET DZVP-MOLOPT-GGA-GTH-q4
      POTENTIAL GTH-PBE-q4
   &END KIND
   &KIND Br
      BASIS_SET DZVP-MOLOPT-GGA-GTH-q7
      POTENTIAL GTH-PBE-q7
   &END KIND
   &KIND H
      BASIS_SET DZVP-MOLOPT-GGA-GTH-q1
      POTENTIAL GTH-PBE-q1
   &END KIND   
&END SUBSYS
&END FORCE_EVAL
"""

## Example of a calculator with Meta-GGA basis sets

In [ ]:
inp_scan="""&FORCE_EVAL
 &DFT
   BASIS_SET_FILE_NAME BASIS_MOLOPT
   BASIS_SET_FILE_NAME BASIS_MOLOPT_UZH
 CHARGE 0
 &QS
   METHOD GPW                       !use gaussians and plane waves
 &END QS
    &SCF
      &OT
        PRECONDITIONER FULL_SINGLE_INVERSE
        MINIMIZER DIIS
        N_DIIS 7
      &END
      SCF_GUESS ATOMIC
      EPS_SCF 2.0E-5
      MAX_SCF 100
       &OUTER_SCF
          EPS_SCF 2.0E-5
          MAX_SCF 100
       &END
    &END SCF
 &END DFT
  &SUBSYS   
   &KIND Cu
     BASIS_SET DZVP-MOLOPT-SCAN-GTH-q11
    POTENTIAL GTH-MGGA-q11
   &END KIND
   &KIND C
      BASIS_SET DZVP-MOLOPT-SCAN-GTH-q4
      POTENTIAL GTH-MGGA-q4
   &END KIND
   &KIND Br
      BASIS_SET DZVP-MOLOPT-SCAN-GTH-q7
      POTENTIAL GTH-MGGA-q7
   &END KIND
   &KIND H
      BASIS_SET DZVP-MOLOPT-SCAN-GTH-q1
      POTENTIAL GTH-MGGA-q1
   &END KIND   
&END SUBSYS
&END FORCE_EVAL
"""

In [ ]:
def ini_calcs():
    global calc_pbed3, calc_scan
    from ase.calculators.cp2k import CP2K
    calc_pbed3 = CP2K(inp=inp_pbed3,command="env OMP_NUM_THREADS=4  /usr/bin/cp2k_shell.psmp",basis_set_file=None,potential_file="POTENTIAL",force_eval_method='QS',basis_set=None,pseudo_potential=None,cutoff=5440,stress_tensor=False,max_scf=None,xc="PBE")
    calc_scan = CP2K(inp=inp_scan,command="env OMP_NUM_THREADS=4  /usr/bin/cp2k_shell.psmp",basis_set_file=None,potential_file="POTENTIAL_UZH",force_eval_method='QS',basis_set=None,pseudo_potential=None,cutoff=5440,stress_tensor=False,max_scf=None,xc="MGGA_C_SCAN_RVV10")
    

In [ ]:
#
# Initialize the calculators
#

ini_calcs()

## First of all, we will create substrates.


## Cu (111)

In [ ]:

custom_lattice_constant = 3.569  # Adjusted from the default, from literature about PBED3 (typically ~3.61 Å)
                                 #  https://onlinelibrary.wiley.com/doi/10.1002/jcc.23037

# Create bulk copper with the modified lattice constant


cu_bulk = bulk('Cu', 'fcc', a=custom_lattice_constant)

# Create Cu(111) slab
cu_slab = surface(cu_bulk, (1, 1, 1), layers=2)

# Expand in x and y directions
cu_slab = cu_slab.repeat((5, 5, 1))

# Check number of atoms
num_atoms = len(cu_slab)
print(f"Number of atoms in the slab: {num_atoms}")


# Increase vacuum only in z-direction
cu_slab.cell[2, 2] += 10  # Adds 10 Å to the z-dimension

cu_slab.set_pbc([True, True, True]) # all periodic for the solution of the Poisson equation 

# Check slab cell dimensions
print("Cell dimensions:", cu_slab.cell)


# Initialize empty nested dictionary

energy ["cu_slab"] = {}

#
# This is the important write. 
#

write ("cu_slab_ini.xyz",cu_slab)

# View slab
view_structure(cu_slab)



In [ ]:
!rm cp2k.out


# Choose the calculator: pbed3 for the optimization of the slab


#cu_slab.calc = None
ini_calcs()
calc ["cu_slab"] = calc_pbed3
cu_slab.calc = calc ["cu_slab"]
print (cu_slab.calc)

myenergy = cu_slab.get_potential_energy()
energy ["cu_slab"]["pbed3"] = myenergy
print(f'Energy of cu_slab: {myenergy} eV')
write ("cu_slab_ini.xyz",cu_slab)


## Au (111)

In [ ]:

custom_lattice_constant = 4.115  # Adjusted from the default, from literature about PBED3 
                                 #  https://onlinelibrary.wiley.com/doi/10.1002/jcc.23037

# Create bulk copper with the modified lattice constant


au_bulk = bulk('Au', 'fcc', a=custom_lattice_constant)

# Create Au(111) slab
au_slab = surface(au_bulk, (1, 1, 1), layers=2)

# Expand in x and y directions
au_slab = au_slab.repeat((5, 5, 1))

# Check number of atoms
num_atoms = len(au_slab)
print(f"Number of atoms in the slab: {num_atoms}")


# Increase vacuum only in z-direction
au_slab.cell[2, 2] += 10  # Adds 10 Å to the z-dimension

au_slab.set_pbc([True, True, True]) # all periodic for the solution of the Poisson equation 

# Check slab cell dimensions
print("Cell dimensions:", au_slab.cell)


# Initialize empty nested dictionary

energy ["au_slab"] = {}



# View slab
view_structure(au_slab)



In [ ]:
!rm cp2k.out


# Choose the calculator: pbed3 for the optimization of the slab


#cu_slab.calc = None
ini_calcs()
calc ["au_slab"] = calc_pbed3
au_slab.calc = calc ["au_slab"]
print (au_slab.calc)

myenergy = au_slab.get_potential_energy()
energy ["au_slab"]["pbed3"] = myenergy
print(f'Energy of au_slab: {myenergy} eV')
write ("au_slab_ini.xyz",au_slab)


# BROMOBENZENE

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
smiles = "C1=CC=C(C=C1)Br"

# Convert SMILES to RDKit molecule
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# Generate 3D conformer
AllChem.EmbedMolecule(mol, AllChem.ETKDG())

# Extract atomic positions
conf = mol.GetConformer()
positions = np.array([conf.GetAtomPosition(i) for i in range(mol.GetNumAtoms())])

# Convert to ASE Atoms
symbols = [atom.GetSymbol() for atom in mol.GetAtoms()]
bromobenzene = Atoms(symbols, positions=positions)

bromobenzene.set_cell([10, 10, 10])
bromobenzene.set_pbc([True,True,True])
bromobenzene.center()

# Initialize nested energy dictionary
energy ["bromobenzene"] = {}
view_structure(bromobenzene)

In [ ]:
distance =  bromobenzene.get_distance(3, 6)
print ("C-Br bond distance: ",distance)

## Let's optimize bromobenzene

In [2]:
## Definition of optimization function
def myoptimize(system,model,level):
    opt = BFGS(system)
    fmax_threshold = 0.05

    while True:
        opt.step()  # Perform a single optimization step
        energy [model][level] = system.get_potential_energy()
        print ("Energy: ",energy [model][level])
        write(model+"_opt.xyz", system, format='extxyz', append=True)  # Append structure
        # Run optimization with proper exit condition

        # Check max force using ASE's built-in functionality
        fmax = max(system.get_forces().flatten())
        print ("Fmax: ",fmax," (",fmax_threshold,")")


        if fmax < fmax_threshold:
            print(f"Optimization stopped: fmax = {fmax:.4f} eV/Å")
            break

    return energy[model][level]


In [ ]:
ini_calcs()
calc["bromobenzene"] = calc_pbed3
bromobenzene.calc = calc_pbed3
# myenergy = bromobenzene.get_potential_energy()
# energy ["bromobenzene"]["pbed3"] = myenergy






In [ ]:

!rm cp2k.out
!rm bromobenzene_opt.xyz 
myoptimize(bromobenzene,"bromobenzene","pbed3")

write ("bromobenzene_ini.xyz",bromobenzene)

In [ ]:
print (energy["bromobenzene"]["pbed3"])

In [ ]:
traj = read ("bromobenzene_opt.xyz",":")
myview=view_trajectory(traj)
myview.add_distance(atom_pair=[[3,6]], label_color="black")
myview

## Adding bromobenzene on the slab

In [ ]:
#for security, I copy the optimized bromobenzene into another instance

bromobenzene_opt = bromobenzene.copy()

In [ ]:

bromobenzene = read("bromobenzene_ini.xyz")

bromobenzene.rotate(30, 'z', center='COM')

#
# one could start with this molecule... but in the next cell I took a geometry from a previously optimized run
#


In [ ]:
from ase.build import add_adsorbate
cu_bromobenzene = cu_slab.copy()
# 
# read from a previous optimization:
#
bromobenzene = read("bromobenzene_oncu.xyz")
cu_bromobenzene = cu_slab.copy()

# Place bromobenzene on the slab
add_adsorbate(cu_bromobenzene, bromobenzene, height=2.9, position=(0,0))



target_coo = (cu_bromobenzene.positions[35]+cu_bromobenzene.positions[27])*0.5

brx = cu_bromobenzene.positions[56,0]
bry = cu_bromobenzene.positions[56,1]
brz = cu_bromobenzene.positions[56,2]



target_coo [2] += 9.505-7.060

displacement_xyz = [target_coo[0]-brx,target_coo[1]-bry,target_coo[2]-brz]


cu_bromobenzene.positions [50:] += displacement_xyz



# Center the slab
cu_bromobenzene.center(vacuum=5.0, axis=2)

# Print structure

if 'adsorbate_info' in cu_bromobenzene.info:
    del cu_bromobenzene.info['adsorbate_info']

cu_bromobenzene.write ("cu_bromobenzene.xyz")

# Initialize nested energy dictionary
energy ["cu_bromobenzene"] = {}

view_structure(cu_bromobenzene)


In [ ]:
#
# fix all the copper atoms during optimization (to fit the computer)
# 

sorted_atoms = sorted(cu_bromobenzene, key=lambda atom: atom.position[2])
for atom in sorted_atoms:
    print(f"Atom {atom.index}: Z = {atom.position[2]:.2f} Å")

    # Fix all 2 layers
bottom_layer_indices = [atom.index for atom in cu_bromobenzene if atom.position[2] < 8]  # Adjust threshold

print (bottom_layer_indices)
constraint = FixAtoms(indices=bottom_layer_indices)
cu_bromobenzene.set_constraint(constraint)

In [ ]:
ini_calcs()
calc ["cu_bromobenzene"] = calc_pbed3
cu_bromobenzene.calc = calc ["cu_bromobenzene"]
print (cu_bromobenzene.calc)

In [ ]:
!rm cp2k.out
!rm cu_bromobenzene_opt*xyz 
system = cu_bromobenzene
model = "cu_bromobenzene"
myoptimize (cu_bromobenzene,"cu_bromobenzene","pbed3")

In [ ]:
write ("cu_bromobenzene_ini.xyz",cu_bromobenzene)

print(energy["cu_bromobenzene"]["pbed3"])

In [ ]:
traj = read ("cu_bromobenzene_opt.xyz",":")
myview=view_trajectory(traj)
myview.add_distance(atom_pair=[[3,6]], label_color="black")
myview

# Debrominated molecule

In [ ]:

cu_debr_bromobenzene = cu_slab.copy()
# 
# read from a previous optimization:
#
debr_bromobenzene = read("debr_bromobenzene_oncu.xyz")

#debr_bromobenzene.rotate (15,v=[-1,2,0],center='COM')
# Place bromobenzene on the slab
add_adsorbate(cu_debr_bromobenzene, debr_bromobenzene, height=2.07, position=(0,0))



target_coo = (cu_debr_bromobenzene.positions[35]+cu_debr_bromobenzene.positions[27]+cu_debr_bromobenzene.positions[25])/3.
brx = cu_debr_bromobenzene.positions[56,0]
bry = cu_debr_bromobenzene.positions[56,1]
brz = cu_debr_bromobenzene.positions[56,2]

# target_coo [2] += 9.505-7.060

displacement_xyz = [target_coo[0]-brx,target_coo[1]-bry,target_coo[2]-brz+2.07]


cu_debr_bromobenzene.positions [50:] += displacement_xyz

cu_debr_bromobenzene.positions [56] += [0.,0.,0.]



# Center the slab
cu_debr_bromobenzene.center(vacuum=5.0, axis=2)

# Print structure

if 'adsorbate_info' in cu_debr_bromobenzene.info:
    del cu_debr_bromobenzene.info['adsorbate_info']

cu_debr_bromobenzene.write ("cu_debr_bromobenzene.xyz")

# Initialize nested energy dictionary
energy ["cu_debr_bromobenzene"] = {}

view_structure(cu_debr_bromobenzene)


In [ ]:

sorted_atoms = sorted(cu_debr_bromobenzene, key=lambda atom: atom.position[2])
#for atom in sorted_atoms:
#    print(f"Atom {atom.index}: Z = {atom.position[2]:.2f} Å")

    # Fix all 2 layers
bottom_layer_indices = [atom.index for atom in cu_debr_bromobenzene if atom.position[2] < 8]  # Adjust threshold

print (bottom_layer_indices)
constraint = FixAtoms(indices=bottom_layer_indices)
cu_debr_bromobenzene.set_constraint(constraint)

In [ ]:
ini_calcs()
calc ["cu_debr_bromobenzene"] = calc_pbed3
cu_debr_bromobenzene.calc = calc ["cu_debr_bromobenzene"]
print (cu_debr_bromobenzene.calc)

In [ ]:
!rm cp2k.out
!rm cu_debr_bromobenzene_opt*xyz 
system = cu_debr_bromobenzene
model = "cu_debr_bromobenzene"
myoptimize (cu_debr_bromobenzene,"cu_debr_bromobenzene","pbed3")

In [ ]:
print (energy ["cu_debr_bromobenzene"]["pbed3"], energy["cu_bromobenzene"]["pbed3"])

# ASSIGNMENT 1: COMPUTE THE ADSORPTION ENERGY OF BROMOBENZENE AND DEBROMINATED BROMOBENZENE ON THE CU SURFACE

# Testing directly the configurations from the paper

## PBE-D3 debromination on Cu


In [ ]:

# 
# read from a reduced geometry taken from Chen et al.:
#
cu_debr_bromobenzene_paper = read("Orig/cu_debr_bromobenzene_paper.xyz")

cu_debr_bromobenzene_paper.cell[2,2] = 15.



# Print structure

if 'adsorbate_info' in cu_debr_bromobenzene_paper.info:
    del cu_debr_bromobenzene_paper.info['adsorbate_info']

cu_debr_bromobenzene_paper.write ("cu_debr_bromobenzene_paper.xyz")

# Initialize nested energy dictionary
energy ["cu_debr_bromobenzene_paper"] = {}

view_structure(cu_debr_bromobenzene_paper)



In [ ]:
ini_calcs()
calc ["cu_debr_bromobenzene_paper"] = calc_pbed3
cu_debr_bromobenzene_paper.calc = calc ["cu_debr_bromobenzene_paper"]
print (cu_debr_bromobenzene_paper.calc)

In [ ]:
energy["cu_debr_bromobenzene_paper"]["pbed3"]=cu_debr_bromobenzene_paper.get_potential_energy()

In [ ]:
print (energy["cu_debr_bromobenzene_paper"]["pbed3"])

## PBE-D3 Bromobenzene on Cu

In [ ]:

# 
# read from a reduced geometry taken from Chen et al.:
#
cu_bromobenzene_paper = read("Orig/cu_bromobenzene_paper.xyz")




# Print structure

if 'adsorbate_info' in cu_bromobenzene_paper.info:
    del cu_bromobenzene_paper.info['adsorbate_info']

cu_bromobenzene_paper.write ("cu_bromobenzene_paper.xyz")

# Initialize nested energy dictionary
energy ["cu_bromobenzene_paper"] = {}

view_structure(cu_bromobenzene_paper)



In [ ]:
ini_calcs()
calc ["cu_bromobenzene_paper"] = calc_pbed3
cu_bromobenzene_paper.calc = calc ["cu_bromobenzene_paper"]
print (cu_bromobenzene_paper.calc)

In [ ]:
energy["cu_bromobenzene_paper"]["pbed3"]=cu_bromobenzene_paper.get_potential_energy()


In [ ]:
print(energy["cu_bromobenzene_paper"]["pbed3"])

## PBE-D3 bromobenzene on Au

In [ ]:

# 
# read from a reduced geometry taken from Chen et al.:
#
au_bromobenzene_paper = read("Orig/au_bromobenzene_paper.xyz")




# Print structure

if 'adsorbate_info' in au_bromobenzene_paper.info:
    del au_bromobenzene_paper.info['adsorbate_info']

au_bromobenzene_paper.write ("au_bromobenzene_paper.xyz")

# Initialize nested energy dictionary
energy ["au_bromobenzene_paper"] = {}

view_structure(au_bromobenzene_paper)


In [ ]:
ini_calcs()
calc ["au_bromobenzene_paper"] = calc_pbed3
au_bromobenzene_paper.calc = calc ["au_bromobenzene_paper"]
print (au_bromobenzene_paper.calc)

In [ ]:
energy["au_bromobenzene_paper"]["pbed3"]=au_bromobenzene_paper.get_potential_energy()
print(energy["au_bromobenzene_paper"]["pbed3"])

## Pbe-D3 Debromination on Au

In [ ]:

# 
# read from a reduced geometry taken from Chen et al.:
#
au_debr_bromobenzene_paper = read("Orig/au_debr_bromobenzene_paper.xyz")

au_debr_bromobenzene_paper.cell[2,2] = 15.



# Print structure

if 'adsorbate_info' in au_debr_bromobenzene_paper.info:
    del au_debr_bromobenzene_paper.info['adsorbate_info']

au_debr_bromobenzene_paper.write ("au_debr_bromobenzene_paper.xyz")

# Initialize nested energy dictionary
energy ["au_debr_bromobenzene_paper"] = {}

view_structure(au_debr_bromobenzene_paper)



In [ ]:
ini_calcs()
calc ["au_debr_bromobenzene_paper"] = calc_pbed3
au_debr_bromobenzene_paper.calc = calc ["au_debr_bromobenzene_paper"]
print (au_debr_bromobenzene_paper.calc)

In [ ]:
energy["au_debr_bromobenzene_paper"]["pbed3"]=au_debr_bromobenzene_paper.get_potential_energy()
print(energy["au_debr_bromobenzene_paper"]["pbed3"])

# ASSIGNMENT: COMPARE THE REACTION ENERGIES ON AU AND CU. DISCUSS THE DIFFERENCES. COMPARE WITH THE VALUES OF THE PAPER.

## SCAN-Rvv10 on Cu

In [ ]:

# 
# read from a reduced geometry taken from Chen et al.:
#
cu_scan_debr_bromobenzene_paper = read("Orig/cu_scan_debr_bromobenzene_paper.xyz")

cu_scan_debr_bromobenzene_paper.cell[2,2] = 15.



# Print structure

if 'adsorbate_info' in cu_scan_debr_bromobenzene_paper.info:
    del cu_scan_debr_bromobenzene_paper.info['adsorbate_info']

cu_scan_debr_bromobenzene_paper.write ("cu_scan_debr_bromobenzene_paper.xyz")

# Initialize nested energy dictionary
energy ["cu_scan_debr_bromobenzene_paper"] = {}

view_structure(cu_scan_debr_bromobenzene_paper)


In [ ]:
ini_calcs()
calc ["cu_scan_debr_bromobenzene_paper"] = calc_scan
cu_scan_debr_bromobenzene_paper.calc = calc ["cu_scan_debr_bromobenzene_paper"]
print (cu_scan_debr_bromobenzene_paper.calc)

In [ ]:
energy["cu_scan_debr_bromobenzene_paper"]["scan"]=cu_scan_debr_bromobenzene_paper.get_potential_energy()
print(energy["cu_scan_debr_bromobenzene_paper"]["scan"])

# ASSIGNMENT: COMPARE RESULT WITH PBE-D3 WITH SCAN. IF THE SCAN-RVV10 DOES NOT CONVERGE, CREATE A CALCULATOR WITH ANOTHER METHOD THAT IS PRESENT IN CP2K 